In [ ]:
# Setting my folder directory
# Spacings are fine!

from os import chdir
path = "C://Users//nelso//Documents//Data Science//Python//Scripts//pySpark"
chdir(path)

In [ ]:
import findspark
findspark.init("C:\devtools\spark-2.4.4-bin-hadoop2.7")
print("Hello! findspark ran fine.")


In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

sc = SparkContext("local", "first app")

# from pyspark.sql import SparkSession

print("SparkContext initialized.")

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Testing PySpark dataframes") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

print('SparkSession Initialized')
df = spark.read.load("sampledf.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

df.select("Recipe").show(5)

In [ ]:
#####Using inbuilt tokenizers

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

tokenizer = Tokenizer(inputCol="Recipe", outputCol="words")
#pipeline = Pipeline(stages=[tokenizer])

wordsData = tokenizer.transform(df.select("Recipe"))
wordsData.show(5)

In [ ]:
#####Using user defined functions
from pyspark.sql.functions import udf
from pyspark.sql.types import *

mywords = ['basil','yeast','buns']

def lowercase(x):
    lowerW = x.lower()
    return lowerW

def matching(x):
    return bool(set(x) & set(mywords))

def find_letters(x):
    first_cap = re.search('[a-z]',x).group(0)
    return first_cap

lowercasing_udf = udf(lowercase, StringType())
matching_udf = udf(matching, BooleanType())
letters_udf = udf(find_letters, StringType())

wordsData2 = df.withColumn('lowercase', lowercasing_udf('Recipe')).select('Recipe','lowercase')
wordsData2.show(5)

wordsData3 = wordsData.withColumn('match',matching_udf('words')).select('Recipe','words','match') \
.withColumn('letters', letters_udf('Recipe')).select('Recipe','words','match','letters')
wordsData3.show(5)

In [ ]:
#####Using user defined functions - from NLTK package 
### THIS SECTION DOESNT WORK
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import re

# word tokenizer
# This will not work, because each slave node does not have the NLTK installed!
import nltk
nltk.data.path.append("C://Users//nelso//Documents//Data Science//Python")
def nltk_tok(x):
    token = nltk.word_tokenize(x)
    return token
print(nltk_tok("Hello!, Big data."))

token_udf = udf(nltk_tok, StringType())

wordsData4 = df.withColumn('tokens', token_udf('Recipe')).select('Recipe','tokens')
wordsData4.show(5)

In [ ]:
with open('mytest.txt') as file:
    myfile = file.read().splitlines()
    
print(myfile)

In [ ]:
from pyspark.sql.types import Row

mydata = sc.textFile('sampledf.csv')

a = mydata \
.map(lambda line: line.split(',')) \
.collect()

print(a)

#df = mydata.toDF()

#df.show(5)

In [ ]:
data = [1,2,3,4,5,6,8,9,10]

# sc = pyspark.SparkContext()
a = sc.parallelize(data).map(lambda x: x*x).collect()
print (a)